# Project: Automated Fact Checking For Climate Science Claims

Student Name: Hongda Zhu

Student ID: 1259524

In [ ]:
import sys
print(sys.version)

# Load datasets

In [ ]:
import json

def load_data(filename):
    file = open(filename, 'r')
    data = json.load(file)
    file.close()
    return data

train_data = load_data("train-claims.json")
dev_data = load_data("dev-claims.json")
evidence_data = load_data("evidence.json")

# print("train_data: ", train_data)
# print("dev_data: ", dev_data)
# print("evidence_data: ", evidence_data)
# for id, info in train_data.items():
#     print("claim_id: ", id)
#     print("claim_text: ", info['claim_text'])
#     print("claim_label: ", info['claim_label'])
#     print("evidences: ", info['evidences'])
#     print()
# print("dev_data: ", dev_data)
# print("evidence_data: ", evidence_data)

# Preprocessing the data

Create PyTorch Dataset and DataLoader for preprocessed data

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

class convertPytorchDataset(Dataset):
    def __init__(self, claim_data, evidence_data, tokenizer, label_mapping):
        self.claim_data = claim_data
        self.evidence_data = evidence_data
        self.tokenizer = tokenizer
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.claim_data)

    def __getitem__(self, idx):
        claim_id, claim_info = list(self.claim_data.items())[idx]
        claim_text = claim_info["claim_text"]
        evidence_texts = [self.evidence_data[evidence_id] for evidence_id in claim_info["evidences"]]
        all_evidence_text = ' '.join(evidence_texts)

        # Turn the data to pytorch tensor, padding them to the same length 
        # If text is too long, truncate them
        tokenized_data = self.tokenizer(claim_text, all_evidence_text, padding=True, truncation=True, return_tensors="pt")

        tokenized_data["claim_id"] = claim_id
        tokenized_data["claim_text"] = claim_info["claim_text"]
        tokenized_data["labels"] = torch.tensor(self.label_mapping[claim_info["claim_label"]])
        tokenized_data["evidences"] = claim_info["evidences"]

        return tokenized_data

# Convert labels to numbers
label_mapping = {
    "SUPPORTS": 0,
    "REFUTES": 1,
    "NOT_ENOUGH_INFO": 2,
    "DISPUTED": 3,
}

train_dataset = convertPytorchDataset(train_data, evidence_data, tokenizer, label_mapping)
dev_dataset = convertPytorchDataset(dev_data, evidence_data, tokenizer, label_mapping)

def collate_fn(batch):
    max_len = max([x["input_ids"].shape[1] for x in batch])
    input_ids = torch.zeros(len(batch), max_len, dtype=torch.long)
    attention_mask = torch.zeros(len(batch), max_len, dtype=torch.long)
    labels = torch.zeros(len(batch), dtype=torch.long)
    claim_ids = []
    claim_texts = []
    evidences = []
    
    for i, x in enumerate(batch):
        cur_len = x["input_ids"].shape[1]
        input_ids[i, :cur_len] = x["input_ids"]
        attention_mask[i, :cur_len] = x["attention_mask"]
        labels[i] = x["labels"]
        claim_ids.append(x["claim_id"])
        claim_texts.append(x["claim_text"])
        evidences.append(x["evidences"])
    
    return {"input_ids": input_ids, "attention_mask": attention_mask, "claim_id": claim_ids, "claim_text": claim_texts, "labels": labels, "evidences": evidences}

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
dev_dataloader = DataLoader(dev_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)


# Training the model

Fine-tune hyperparameter

In [ ]:
search_space = {
    'learning_rate': [1e-4, 3e-5, 1e-5],
    'batch_size': [8, 16, 32],
    'epochs': [2, 3, 4]
}

def train_and_evaluate(params):
    batch_size = params['batch_size']
    lr = params['learning_rate']
    epochs = params['epochs']

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

    for epoch in range(epochs):
        train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)

    val_accuracy = evaluate(model, dev_dataloader, device)

    return val_accuracy


# import itertools

# best_val_accuracy = 0
# best_params = None

# for lr, batch_size, epochs in itertools.product(search_space['learning_rate'], search_space['batch_size'], search_space['epochs']):
#     params = {'learning_rate': lr, 'batch_size': batch_size, 'epochs': epochs}
#     print(f"Trying parameters: {params}")
#     val_accuracy = train_and_evaluate(params)
#     print(f"Validation Accuracy: {val_accuracy:.4f}")

#     if val_accuracy > best_val_accuracy:
#         best_val_accuracy = val_accuracy
#         best_params = params

# print(f"Best hyperparameters: {best_params}")
# print(f"Best validation accuracy: {best_val_accuracy:.4f}")



In [ ]:
from transformers import BertForSequenceClassification

num_labels = len(label_mapping)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

# If gpu available, use gpu.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


from transformers import AdamW, get_linear_schedule_with_warmup

epochs = 3
lr = 3e-5

optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

from tqdm import tqdm

def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)
    print(f"Train Loss: {train_loss:.4f}")


Free GPU memory

In [ ]:
import torch

# Clear the GPU cache
torch.cuda.empty_cache()

from tensorflow.keras import backend as K

# Clear the GPU memory
K.clear_session()

# Prediction on dev data

In [ ]:
label_mapping_inverse = {v: k for k, v in label_mapping.items()}

def save_predictions(model, dataloader, output_file):
    model.eval()
    predictions = {}

    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        claim_ids = batch["claim_id"]
        claim_texts = batch["claim_text"]

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            _, predicted = torch.max(outputs.logits, 1)

        for idx, claim_id in enumerate(claim_ids):
            predictions[claim_id] = {
                "claim_text": claim_texts[idx],
                "claim_label": label_mapping_inverse[predicted[idx].item()],
                "evidences": batch["evidences"][idx]
            }

    with open(output_file, "w") as outfile:
        json.dump(predictions, outfile, indent=2)

save_predictions(model, dev_dataloader, "dev_claims_out.json")

